<a href="https://colab.research.google.com/github/Xuehui-Jiang/Uni-Mannheim-Master-Thesis/blob/main/Experimental_Evaluation/Document_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cloning the pyRDF2Vec repo
!git clone https://github.com/IBCNServices/pyRDF2Vec.git

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 7457, done.
remote: Counting objects: 100% (425/425), done.
remote: Compressing objects: 100% (254/254), done.
remote: Total 7457 (delta 268), reused 315 (delta 164), pack-reused 7032
Receiving objects: 100% (7457/7457), 5.42 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (5072/5072), done.


In [2]:
!pip uninstall community
!pip install python-louvain
!pip install pyRDF2Vec --upgrade

Found existing installation: community 1.0.0b1
Uninstalling community-1.0.0b1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/community-1.0.0b1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/community/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/community/community_louvain.py
    /usr/local/lib/python3.10/dist-packages/community/community_status.py
Proceed (Y/n)? Y
  Successfully uninstalled community-1.0.0b1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install rdflib
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [9]:
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import itertools
from typing import List, Sequence, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rdflib
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, confusion_matrix

import sys
sys.path.append('pyRDF2Vec')
sys.path.append('pyRDF2Vec/pyrdf2vec')

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.walkers import RandomWalker

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


Saving LP50_averageScores.csv to LP50_averageScores.csv
Saving LP50_entities.json to LP50_entities.json
User uploaded file "LP50_averageScores.csv" with length 12725 bytes
User uploaded file "LP50_entities.json" with length 93578 bytes


In [11]:
# Step 1.1: Load Gold Standard Data
gold_standard_df = pd.read_csv('LP50_averageScores.csv')
gold_standard_df.head()

,doc1,doc2,average
0,1,2,1.5
1,1,3,1.2
2,1,4,1.0
3,1,5,1.5
4,1,6,2.5


In [12]:
#Create DataDrame 'data' with entity name as first column, doc column storing which document
#(indicated by document ID) these entities belong to as second column.
#weight column storing the weight of each entity as the third column.
import json

# Load the LP50_entities.json file
with open("LP50_entities.json", "r") as f:
    lp50_entities_data = json.load(f)

# Initialize empty lists to hold the DataFrame columns
name_list = []
doc_list = []
weight_list = []

# Loop through each document in the JSON data
for doc_id, doc_info in enumerate(lp50_entities_data):
    annotations = doc_info['annotations']
    for annotation in annotations:
        name_list.append(annotation['entity'])
        doc_list.append(doc_id + 1)  # Document IDs start from 1
        weight_list.append(annotation['weight'])

# Create DataFrame
data = pd.DataFrame({
    'name': name_list,
    'doc': doc_list,
    'weight': weight_list
})

data.head()

,name,doc,weight
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193
1,http://dbpedia.org/resource/Australian_Senate,1,0.469
2,http://dbpedia.org/resource/Brian_Greig,1,1.000
3,http://dbpedia.org/resource/Interim_leader,1,0.389
4,http://dbpedia.org/resource/Interim,1,0.258


In [13]:
# Load the LP50_entities.json file
with open("LP50_entities.json", "r") as f:
    lp50_entities_data = json.load(f)

# Extract all unique entities from the LP50_entities.json file
unique_entities = set()

# Loop through each document's annotations to collect entities
for doc in lp50_entities_data:
    for annotation in doc['annotations']:
        unique_entities.add(annotation['entity'])

# Convert the set back to a list
unique_entities = list(unique_entities)

# Show the first few unique entities for verification and the total number of unique entities
unique_entities[:10]

['http://dbpedia.org/resource/International_sanctions',
 'http://dbpedia.org/resource/Europe',
 'http://dbpedia.org/resource/Fruit',
 'http://dbpedia.org/resource/General_officer',
 'http://dbpedia.org/resource/Missile',
 'http://dbpedia.org/resource/Starvation',
 'http://dbpedia.org/resource/Russia',
 'http://dbpedia.org/resource/Habbush_letter',
 'http://dbpedia.org/resource/Marriage',
 'http://dbpedia.org/resource/Environmental_degradation']

In [10]:
print(len(unique_entities))

401


In [ ]:
# Filter out entities and relationships directly related to LP50 Dataset
with open("/content/drive/My Drive/dbpedia_files/direct_merged_dbpedia.ttl", "r") as file, open("direct_LP50_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2:
            # Check if the subject or object of the triple is in our LP50 URL list
            if triple[0].strip('<>') in unique_entities or triple[2].strip('<>') in unique_entities:
                out_file.write(line)

In [ ]:
# Filter out entities and relationships directly or indirectly related to LP50 Dataset
entities_related_to_LP50 = set(unique_entities)
new_entities = set(unique_entities)
iteration_count = 0

while new_entities and iteration_count < 2:
    temp_entities = set()
    with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file:
        for line in file:
            triple = line.strip().split()
            if len(triple) > 2 and (triple[0].strip('<>') in new_entities or triple[2].strip('<>') in new_entities):
                    temp_entities.add(triple[0].strip('<>'))
                    temp_entities.add(triple[2].strip('<>'))

    new_entities = temp_entities - entities_related_to_LP50
    entities_related_to_LP50.update(new_entities)
    iteration_count += 1

# Finally, we can write all triples that are directly or indirectly related to entities in the `entities_related_to_LP50` collection
with open("/content/drive/My Drive/dbpedia_files/indirect_merged_dbpedia.ttl", "r") as file, open("indirect_LP50_filtered_dbpedia.ttl", "w") as out_file:
    for line in file:
        triple = line.strip().split()
        if len(triple) > 2 and (triple[0].strip('<>') in entities_related_to_LP50 or triple[2].strip('<>') in entities_related_to_LP50):
            out_file.write(line)


In [ ]:
# Define the list of files
files_to_merge = [
    "direct_LP50_filtered_dbpedia.ttl",
    "indirect_LP50_filtered_dbpedia.ttl"
]

# Merge files
with open("LP50_filtered_dbpedia.ttl", "w") as outfile:
    for file in files_to_merge:
        with open(file, "r") as infile:
            for line in infile:
                outfile.write(line)

print("Files have been merged into merged.ttl")


Files have been merged into merged.ttl


In [ ]:
input_file = "LP50_filtered_dbpedia.ttl"
output_file = "cleaned_LP50_filtered_dbpedia.ttl"

with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Determine whether it is a comment or a blank line
        if line.startswith("#") or line.strip() == "":
            continue

        # 分割三元组
        triple = line.strip().split()

        # Split triples
        if len(triple) > 2:
            # Check whether the object starts with "http://dbpedia.org/resource/"
            if triple[2].startswith("<http://dbpedia.org/resource/"):
                outfile.write(line)

print(f"Finished filtering {input_file} and saved the result to {output_file}")


Finished filtering LP50_filtered_dbpedia.ttl and saved the result to cleaned_LP50_filtered_dbpedia.ttl


In [ ]:
!head -n 10 cleaned_LP50_filtered_dbpedia.ttl

<http://dbpedia.org/resource/Afroasiatic_languages> <http://dbpedia.org/property/region> <http://dbpedia.org/resource/Middle_East> .
<http://dbpedia.org/resource/Alaska> <http://dbpedia.org/property/south> <http://dbpedia.org/resource/Pacific_Ocean> .
<http://dbpedia.org/resource/Angola> <http://dbpedia.org/property/nationalLanguages> <http://dbpedia.org/resource/Kongo_language> .
<http://dbpedia.org/resource/Angola> <http://dbpedia.org/property/nationalLanguages> <http://dbpedia.org/resource/Chokwe_language> .
<http://dbpedia.org/resource/Angola> <http://dbpedia.org/property/nationalLanguages> <http://dbpedia.org/resource/Umbundu> .
<http://dbpedia.org/resource/Angola> <http://dbpedia.org/property/nationalLanguages> <http://dbpedia.org/resource/Kimbundu> .
<http://dbpedia.org/resource/Angola> <http://dbpedia.org/property/nationalLanguages> <http://dbpedia.org/resource/Ganguela_language> .
<http://dbpedia.org/resource/Angola> <http://dbpedia.org/property/nationalLanguages> <http://dbpe

In [ ]:
import shutil

destination_path = "/content/drive/My Drive/dbpedia_files"

shutil.copy("cleaned_LP50_filtered_dbpedia.ttl", destination_path)

print(f"File saved to {destination_path}")

File saved to /content/drive/My Drive/dbpedia_files


In [11]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("/content/drive/My Drive/dbpedia_files/cleaned_LP50_filtered_dbpedia.ttl", fmt='turtle')

kgentities = kg._entities

In [12]:
for e in unique_entities:
    found = False
    for ekg in kgentities:
        s = ekg.name
        if s==e:
            found = True
    if not found:
        print(e)

http://dbpedia.org/resource/Arne_Rinnan
http://dbpedia.org/resource/Defence_Signals_Directorate
http://dbpedia.org/resource/Mandatory_detention_in_Australia
http://dbpedia.org/resource/Brant_Goose


In [14]:
# Entities not in KG
absent_entities = [
    "http://dbpedia.org/resource/Arne_Rinnan",
    "http://dbpedia.org/resource/Defence_Signals_Directorate",
    "http://dbpedia.org/resource/Mandatory_detention_in_Australia",
    "http://dbpedia.org/resource/Brant_Goose"
]

# Filter out absent entities
filtered_entities = [e for e in unique_entities if e not in absent_entities]

In [15]:
from scipy.stats import spearmanr
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances

In [16]:
float_precision = 15

In [17]:
def train(data, stats, with_weights=True, debugging_mode=True, distance_metric='cosine'):
    """
    It trains the model based on the provided data

    data: DataFrame with entity name as first column, doc column storing which document
    (indicated by document ID) these entities belong to as second column and weight column storing
    the weight of each entity as the third column.
    stats: it contains the data used as gold standard
    with_weights: {TRUE, FALSE} if the evaluation metric should consider the weights provided by the annotator or not
    debugging_mode: {TRUE, FALSE}, TRUE to run the model by reporting all the errors and information; FALSE otherwise


    It returns the result object reporting the task name, the used configuration and the evaluation metrics.
    """
    doc_similarity = compute_doc_distance(data, with_weights=with_weights, debugging_mode=debugging_mode, distance_metric=distance_metric)
    gold_similarity_score, similarity_score = get_gold_and_actual_score(
        stats, doc_similarity
    )
    (
        pearson_score,
        spearman_score,
        harmonic_mean,
    ) = evaluate_document_similarity(gold_similarity_score, similarity_score)

    if with_weights:
        conf = "with_weights"
    else:
        conf = "without_weights"

    return {
        "task_name": 'DocumentSimilarity',
        "conf": conf,
        "pearson_score": round(pearson_score, float_precision),
        "spearman_score": round(spearman_score, float_precision),
        "harmonic_mean": round(harmonic_mean, float_precision),
    }

In [18]:
def compute_doc_distance(data, with_weights=True, debugging_mode=True, distance_metric='cosine'):
    """
    It computes the predicted document distance

    data: DataFrame with entity name as first column, doc column storing which document
    (indicated by document ID) these entities belong to as second column and weight column storing
    the weight of each entity as the third column.
    with_weights: {TRUE, FALSE} if the evaluation metric should consider the weights provided by the annotator or not
    debugging_mode: {TRUE, FALSE}, TRUE to run the model by reporting all the errors and information; FALSE otherwise

    It returns
    	the dataframe containing the similarity for each pair of documents;
    """
    result_dict = {}
    doc1_list = list()
    doc2_list = list()
    doc_similarity = list()

    for i in range(1, 51):
        # 1. extract entities of document i and j
        set1 = extract_entities(i, data)
        if len(set1) == 0:
            continue

        for j in range(i, 51):
            set2 = extract_entities(j, data)
            if len(set2) == 0:
                continue

            # 1. DONE : set 1 and set 2 contain entities of document i and j

            # 2. compute the similarity for each pair of entities in d_i and d_j
            # similarity is interpreted as the opposite of distance
            similarity_score1 = compute_similarity(set1, set2, distance_metric=distance_metric)
            similarity_score2 = compute_similarity(set2, set1, distance_metric=distance_metric)
            if with_weights:  # if weights are enabled - multiply distances with weight matrix
                similarity_score1 = similarity_score1 * np.outer(
                    set1["weight"], set2["weight"]
                )
                similarity_score2 = similarity_score2 * np.outer(
                    set2["weight"], set1["weight"]
                )

            # 3. for each entity in d_i identify the maximum similarity to an entity in d2, and viceversa
            max_sim1 = np.max(similarity_score1, axis=1)
            max_sim2 = np.max(similarity_score2, axis=1)

            # 4. calculate document similarity
            document_similarity = (sum(max_sim1) + sum(max_sim2)) / (
                len(max_sim1) + len(max_sim2)
            )

            if debugging_mode:
                print(
                    "Doc "
                    + str(i)
                    + " - Doc "
                    + str(j)
                    + " : distance similarity "
                    + str(document_similarity)
                )

            doc1_list.append(i)
            doc2_list.append(j)
            doc_similarity.append(document_similarity)

    result_dict["doc1"] = doc1_list
    result_dict["doc2"] = doc2_list
    result_dict["similarity"] = doc_similarity

    return pd.DataFrame(result_dict)

In [19]:
def get_gold_and_actual_score(gold_stats, actual_stats):
    """
    It returns the document similarity value used as gold standard and the actual one

    gold_stats: dataframe containing data used as gold standard
    actual_stats: dataframe containing the predicted results
    """
    merged = pd.merge(gold_stats, actual_stats, on=["doc1", "doc2"], how="inner")
    return (merged.iloc[:, 2], merged.iloc[:, 3])

In [20]:
def evaluate_document_similarity(gold_similarity_score, similarity_score):
    """
    It evaluates the predicted document similarity against the document similarity used as gold standard.

    gold_similarity_score: array containing the document similarity used as gold standard
    similarity_score: array containing the actual document similarity score
    """
    spearman_score, _value = spearmanr(gold_similarity_score, similarity_score)
    pearson_score, _value = pearsonr(gold_similarity_score, similarity_score)
    harmonic_mean = (2 * pearson_score * spearman_score) / (
        pearson_score + spearman_score
    )
    return pearson_score, spearman_score, harmonic_mean

In [21]:
def extract_entities(documentID, data, debugging_mode=True):
    """
    It extracts the entities related to the document in input from the whole entities dataframe

    documentID: current document ID as integer from 1 to 50
    data: DataFrame with entity name as first column, doc column storing which document
    (indicated by document ID) these entities belong to as second column and weight column storing
    the weight of each entity as the third column.
    debugging_mode: {TRUE, FALSE}, TRUE to run the model by reporting all the errors and information; FALSE otherwise
    """
    set1 = data[data["doc"] == documentID]
    if len(set1) == 0:
        if debugging_mode:
            print("No entities in doc " + str(documentID))
    else:
        set1 = set1.sort_values("weight", ascending=False).drop_duplicates(
            subset="name", keep="first"
        )
    return set1

In [2]:
def compute_similarity(set1, set2, distance_metric):  #distance_metric='cosine'/'euclidean'
    """
    It computes the similarity among all the entities in the provided inputs.

    set1, set2: dataframe containing the entities of a document

    It returns the pairwise distance of all the pairs in the dataframes provided in input
    """
    return 1 - pairwise_distances(
        set1.iloc[:, 3:], set2.iloc[:, 3:], metric=distance_metric
    )

When use distance_metric='euclidean' with weight

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # Add vectors into DataFrame 'data'
        # Convert the values of the dictionary to a list
        default_value = np.full((dim,), np.nan)
        embedding_list = [embedding_dict.get(entity, default_value) for entity in data['name']]

        embedding_df = pd.DataFrame(embedding_list)

        # Concatenate the original 'data' DataFrame with the 'embedding_df' DataFrame
        data_with_embeddings = pd.concat([data, embedding_df], axis=1)

        # Rename the new column
        new_columns = ['v' + str(i) for i in range(embedding_df.shape[1])]
        col_names = list(data.columns) + new_columns
        data_with_embeddings.columns = col_names
        # Fill in missing entity vectors using the average of other entity vectors within the same document
        # Find out which documents have entities with missing vectors
        docs_with_missing = data_with_embeddings[data_with_embeddings.isnull().any(axis=1)]['doc'].unique()

        for doc in docs_with_missing:
            # Get all entity data in the document
            doc_data = data_with_embeddings[data_with_embeddings['doc'] == doc]

            # Calculate the average of non-missing entity vectors in the document
            average_vector = doc_data.dropna().iloc[:, 3:].mean()

            # Find the entity with the missing vector in the document
            missing_rows = (data_with_embeddings['doc'] == doc) & data_with_embeddings.isnull().any(axis=1)

            # Fill missing vectors with average vector
            data_with_embeddings.loc[missing_rows, new_columns] = average_vector.values
            display(data_with_embeddings)

        # 3. Call the Document Similarity evaluation method
        result_dict = train(data_with_embeddings, gold_standard_df, with_weights=True, debugging_mode=False, distance_metric='euclidean')

        # 4. Collect and save results
        results.append({
            'Embedding Mode': mode,
            'Dimensions': dim,
            'conf': result_dict['conf'],
            'Pearson Score': result_dict['pearson_score'],
            'Spearman Score': result_dict['spearman_score'],
            'Harmonic Mean': result_dict['harmonic_mean']
})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('document_similarity_scores_eu_we.csv', index=False)


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,0.787165,-0.948554,3.126753,-1.110511,2.054212,0.814279,2.707326,...,1.365170,-0.346776,1.373162,-2.438210,1.134281,1.034633,2.244160,-2.860556,1.438885,0.070871
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,2.816050,-2.672302,4.456600,-1.039608,1.507375,-0.405454,2.254813,...,-0.362333,2.936858,2.842382,-0.160649,-0.208341,-0.915376,-0.732123,-3.218554,-1.413494,2.942373
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.805877,-1.447392,-0.068966,2.141130,0.758985,-0.467688,1.924895,...,2.900114,0.399210,-0.413635,-0.954852,1.568123,-3.062001,1.194655,-2.344593,-0.224452,0.971343
3,http://dbpedia.org/resource/Interim_leader,1,0.389,-0.106146,-0.129593,0.284296,0.199506,0.174440,-0.185512,0.032377,...,0.001735,0.060792,0.028236,-0.093949,0.193967,-0.106005,0.119780,-0.054357,0.031307,0.206896
4,http://dbpedia.org/resource/Interim,1,0.258,-0.040810,-0.054314,0.153887,0.093938,0.114674,-0.129911,0.016041,...,-0.011714,0.062661,0.001421,-0.051726,0.130147,-0.111395,0.068429,-0.038638,0.048858,0.107123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,1.039161,-0.503248,1.195473,2.226826,0.917403,0.492876,1.638808,...,-0.811492,0.290459,1.618784,-0.349413,0.201438,-1.391304,2.151477,-1.909178,-1.011077,1.875672
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,-0.005170,0.181711,0.076120,-0.037859,0.271027,-0.481461,0.235891,...,0.100698,0.158697,-0.072095,0.003981,0.538285,0.080570,0.457394,-0.300689,0.077870,0.274030
635,http://dbpedia.org/resource/Refugee,50,0.507,-0.070328,0.292586,0.462761,0.191388,0.437649,-0.732493,0.396443,...,0.045602,0.237803,-0.152625,0.074847,0.940446,-0.092621,0.780701,-0.450701,0.140840,0.390904
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,0.787165,-0.948554,3.126753,-1.110511,2.054212,0.814279,2.707326,...,1.365170,-0.346776,1.373162,-2.438210,1.134281,1.034633,2.244160,-2.860556,1.438885,0.070871
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,2.816050,-2.672302,4.456600,-1.039608,1.507375,-0.405454,2.254813,...,-0.362333,2.936858,2.842382,-0.160649,-0.208341,-0.915376,-0.732123,-3.218554,-1.413494,2.942373
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.805877,-1.447392,-0.068966,2.141130,0.758985,-0.467688,1.924895,...,2.900114,0.399210,-0.413635,-0.954852,1.568123,-3.062001,1.194655,-2.344593,-0.224452,0.971343
3,http://dbpedia.org/resource/Interim_leader,1,0.389,-0.106146,-0.129593,0.284296,0.199506,0.174440,-0.185512,0.032377,...,0.001735,0.060792,0.028236,-0.093949,0.193967,-0.106005,0.119780,-0.054357,0.031307,0.206896
4,http://dbpedia.org/resource/Interim,1,0.258,-0.040810,-0.054314,0.153887,0.093938,0.114674,-0.129911,0.016041,...,-0.011714,0.062661,0.001421,-0.051726,0.130147,-0.111395,0.068429,-0.038638,0.048858,0.107123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,1.039161,-0.503248,1.195473,2.226826,0.917403,0.492876,1.638808,...,-0.811492,0.290459,1.618784,-0.349413,0.201438,-1.391304,2.151477,-1.909178,-1.011077,1.875672
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,-0.005170,0.181711,0.076120,-0.037859,0.271027,-0.481461,0.235891,...,0.100698,0.158697,-0.072095,0.003981,0.538285,0.080570,0.457394,-0.300689,0.077870,0.274030
635,http://dbpedia.org/resource/Refugee,50,0.507,-0.070328,0.292586,0.462761,0.191388,0.437649,-0.732493,0.396443,...,0.045602,0.237803,-0.152625,0.074847,0.940446,-0.092621,0.780701,-0.450701,0.140840,0.390904
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,0.787165,-0.948554,3.126753,-1.110511,2.054212,0.814279,2.707326,...,1.365170,-0.346776,1.373162,-2.438210,1.134281,1.034633,2.244160,-2.860556,1.438885,0.070871
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,2.816050,-2.672302,4.456600,-1.039608,1.507375,-0.405454,2.254813,...,-0.362333,2.936858,2.842382,-0.160649,-0.208341,-0.915376,-0.732123,-3.218554,-1.413494,2.942373
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.805877,-1.447392,-0.068966,2.141130,0.758985,-0.467688,1.924895,...,2.900114,0.399210,-0.413635,-0.954852,1.568123,-3.062001,1.194655,-2.344593,-0.224452,0.971343
3,http://dbpedia.org/resource/Interim_leader,1,0.389,-0.106146,-0.129593,0.284296,0.199506,0.174440,-0.185512,0.032377,...,0.001735,0.060792,0.028236,-0.093949,0.193967,-0.106005,0.119780,-0.054357,0.031307,0.206896
4,http://dbpedia.org/resource/Interim,1,0.258,-0.040810,-0.054314,0.153887,0.093938,0.114674,-0.129911,0.016041,...,-0.011714,0.062661,0.001421,-0.051726,0.130147,-0.111395,0.068429,-0.038638,0.048858,0.107123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,1.039161,-0.503248,1.195473,2.226826,0.917403,0.492876,1.638808,...,-0.811492,0.290459,1.618784,-0.349413,0.201438,-1.391304,2.151477,-1.909178,-1.011077,1.875672
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,-0.005170,0.181711,0.076120,-0.037859,0.271027,-0.481461,0.235891,...,0.100698,0.158697,-0.072095,0.003981,0.538285,0.080570,0.457394,-0.300689,0.077870,0.274030
635,http://dbpedia.org/resource/Refugee,50,0.507,-0.070328,0.292586,0.462761,0.191388,0.437649,-0.732493,0.396443,...,0.045602,0.237803,-0.152625,0.074847,0.940446,-0.092621,0.780701,-0.450701,0.140840,0.390904
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,0.537614,-0.700272,1.118433,0.323835,0.477639,-0.351009,0.900423,...,0.078572,0.742808,0.919791,-0.339372,0.814253,-0.314955,0.805413,-1.079694,-0.884008,0.914757


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,-2.320138,0.437114,-0.398529,0.991630,2.231609,0.505656,-1.164617,...,-0.741512,0.380678,0.909642,-0.963383,0.106099,1.858457,-1.618444,0.084205,0.011199,0.499584
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,-0.963238,-1.083990,1.460269,0.587880,1.938950,0.376754,-3.245754,...,-2.030996,-0.866047,-0.141696,-0.381765,0.681638,0.491712,-2.972728,-0.479568,0.734883,0.029085
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.140593,-0.168557,-0.135014,2.713067,1.347679,0.637453,-0.787543,...,-0.048231,-0.356130,0.232825,-0.486017,-0.612958,0.036571,-0.421619,-0.178736,0.530872,0.804436
3,http://dbpedia.org/resource/Interim_leader,1,0.389,0.068190,0.030319,-0.070913,0.124622,0.008975,-0.013205,-0.016372,...,-0.066514,0.093061,0.205928,-0.006445,0.051400,0.001775,0.093986,0.026702,0.163417,0.044444
4,http://dbpedia.org/resource/Interim,1,0.258,0.056588,0.007238,-0.036368,0.066855,0.033230,0.008941,-0.023914,...,-0.035887,0.055710,0.128105,-0.006271,0.001197,0.002122,0.046246,0.043780,0.089982,0.015834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,-0.442274,0.413381,-0.545320,1.408084,1.245481,0.691582,-1.367540,...,-1.283408,0.126124,0.403890,0.046618,0.856622,-0.879044,-0.780066,-1.438576,0.991186,0.394975
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,-0.168696,0.250203,-0.280377,-0.135673,0.143133,0.127305,0.176380,...,-0.219998,0.315588,0.336378,-0.209354,0.090081,0.128800,0.163591,-0.081405,0.121421,-0.115143
635,http://dbpedia.org/resource/Refugee,50,0.507,-0.275403,0.400011,-0.543122,-0.045101,0.220713,0.311332,0.260231,...,-0.376424,0.450980,0.598929,-0.294767,-0.014499,0.229933,0.323572,0.072617,0.193546,-0.155146
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,-2.320138,0.437114,-0.398529,0.991630,2.231609,0.505656,-1.164617,...,-0.741512,0.380678,0.909642,-0.963383,0.106099,1.858457,-1.618444,0.084205,0.011199,0.499584
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,-0.963238,-1.083990,1.460269,0.587880,1.938950,0.376754,-3.245754,...,-2.030996,-0.866047,-0.141696,-0.381765,0.681638,0.491712,-2.972728,-0.479568,0.734883,0.029085
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.140593,-0.168557,-0.135014,2.713067,1.347679,0.637453,-0.787543,...,-0.048231,-0.356130,0.232825,-0.486017,-0.612958,0.036571,-0.421619,-0.178736,0.530872,0.804436
3,http://dbpedia.org/resource/Interim_leader,1,0.389,0.068190,0.030319,-0.070913,0.124622,0.008975,-0.013205,-0.016372,...,-0.066514,0.093061,0.205928,-0.006445,0.051400,0.001775,0.093986,0.026702,0.163417,0.044444
4,http://dbpedia.org/resource/Interim,1,0.258,0.056588,0.007238,-0.036368,0.066855,0.033230,0.008941,-0.023914,...,-0.035887,0.055710,0.128105,-0.006271,0.001197,0.002122,0.046246,0.043780,0.089982,0.015834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,-0.442274,0.413381,-0.545320,1.408084,1.245481,0.691582,-1.367540,...,-1.283408,0.126124,0.403890,0.046618,0.856622,-0.879044,-0.780066,-1.438576,0.991186,0.394975
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,-0.168696,0.250203,-0.280377,-0.135673,0.143133,0.127305,0.176380,...,-0.219998,0.315588,0.336378,-0.209354,0.090081,0.128800,0.163591,-0.081405,0.121421,-0.115143
635,http://dbpedia.org/resource/Refugee,50,0.507,-0.275403,0.400011,-0.543122,-0.045101,0.220713,0.311332,0.260231,...,-0.376424,0.450980,0.598929,-0.294767,-0.014499,0.229933,0.323572,0.072617,0.193546,-0.155146
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...

,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,0.215088,-0.230579,0.633413,0.264283,0.339900,-0.311941,0.063349,...,-0.205450,0.235032,0.504806,-0.046753,-0.079424,-0.414342,-0.740460,-0.202931,0.501586,-0.752228
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,0.555385,-0.110233,0.072655,0.578902,0.058908,0.545115,0.098495,...,-0.050912,-0.236830,-0.153634,-0.395646,-0.117633,-0.719989,-0.280392,-0.120660,-0.692730,-0.753878
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,0.017973,0.252894,0.265161,0.485079,-0.230452,0.013451,0.115603,...,-0.324459,-0.031734,0.515665,-0.277305,0.621153,-0.017446,-0.122088,-0.318772,-0.505090,-0.804580
3,http://dbpedia.org/resource/Interim_leader,1,0.389,-0.080772,0.086005,-0.062791,0.123559,-0.155117,0.087060,-0.012665,...,0.033325,0.023613,-0.007818,0.005763,0.107536,-0.053155,0.032890,-0.040521,-0.031400,-0.126006
4,http://dbpedia.org/resource/Interim,1,0.258,-0.058549,0.064420,-0.031483,0.098133,-0.092722,0.048043,-0.015955,...,0.019143,0.018984,-0.022619,-0.033698,0.088643,-0.031840,-0.033889,-0.041679,-0.023462,-0.129770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,0.676295,0.288385,0.152225,0.146346,0.110349,0.312107,0.339327,...,-0.152000,-0.297105,0.226675,-0.534679,-0.101880,-0.201899,-0.144049,-0.371979,-0.823987,-0.784159
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,0.038453,0.210434,-0.110680,0.117303,-0.413682,0.250778,-0.102139,...,0.041797,0.004475,0.034230,0.065842,-0.058793,0.053170,0.076752,-0.007236,0.083053,-0.034570
635,http://dbpedia.org/resource/Refugee,50,0.507,0.100761,0.088431,-0.081908,0.155296,-0.441265,0.169385,-0.263406,...,0.113358,0.053754,-0.034063,-0.016048,-0.075118,0.081345,-0.050911,0.085201,0.021731,0.084123
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,0.215088,-0.230579,0.633413,0.264283,0.339900,-0.311941,0.063349,...,-0.205450,0.235032,0.504806,-0.046753,-0.079424,-0.414342,-0.740460,-0.202931,0.501586,-0.752228
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,0.555385,-0.110233,0.072655,0.578902,0.058908,0.545115,0.098495,...,-0.050912,-0.236830,-0.153634,-0.395646,-0.117633,-0.719989,-0.280392,-0.120660,-0.692730,-0.753878
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,0.017973,0.252894,0.265161,0.485079,-0.230452,0.013451,0.115603,...,-0.324459,-0.031734,0.515665,-0.277305,0.621153,-0.017446,-0.122088,-0.318772,-0.505090,-0.804580
3,http://dbpedia.org/resource/Interim_leader,1,0.389,-0.080772,0.086005,-0.062791,0.123559,-0.155117,0.087060,-0.012665,...,0.033325,0.023613,-0.007818,0.005763,0.107536,-0.053155,0.032890,-0.040521,-0.031400,-0.126006
4,http://dbpedia.org/resource/Interim,1,0.258,-0.058549,0.064420,-0.031483,0.098133,-0.092722,0.048043,-0.015955,...,0.019143,0.018984,-0.022619,-0.033698,0.088643,-0.031840,-0.033889,-0.041679,-0.023462,-0.129770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,0.676295,0.288385,0.152225,0.146346,0.110349,0.312107,0.339327,...,-0.152000,-0.297105,0.226675,-0.534679,-0.101880,-0.201899,-0.144049,-0.371979,-0.823987,-0.784159
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,0.038453,0.210434,-0.110680,0.117303,-0.413682,0.250778,-0.102139,...,0.041797,0.004475,0.034230,0.065842,-0.058793,0.053170,0.076752,-0.007236,0.083053,-0.034570
635,http://dbpedia.org/resource/Refugee,50,0.507,0.100761,0.088431,-0.081908,0.155296,-0.441265,0.169385,-0.263406,...,0.113358,0.053754,-0.034063,-0.016048,-0.075118,0.081345,-0.050911,0.085201,0.021731,0.084123
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,0.156016,0.147076,0.005767,0.151779,-0.166442,0.186839,-0.021349,...,-0.046125,0.002562,0.076159,-0.133974,0.083957,-0.161751,-0.132389,-0.234302,-0.203428,-0.347401


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,-0.084761,-0.116680,0.550538,0.560814,-0.466635,0.195705,0.292799,...,0.402659,0.264922,-0.218491,0.099624,0.019939,0.105649,-0.142350,0.232590,-0.296790,0.215688
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,0.306742,-0.131736,-0.299866,0.116574,-0.588413,0.145954,0.314122,...,0.287308,-0.034254,0.501102,0.331282,-0.170772,0.149103,-0.566943,-0.132967,-0.179744,0.470595
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.127127,0.095752,-0.285491,-0.109764,-0.221885,0.246013,0.509733,...,0.311846,0.252313,-0.120391,0.108980,-0.078103,0.004863,-0.008975,-0.132159,-0.052691,-0.052647
3,http://dbpedia.org/resource/Interim_leader,1,0.389,0.141770,-0.122628,-0.024322,-0.014976,-0.039883,-0.026032,0.039611,...,0.151385,-0.069491,-0.016031,0.015915,0.038179,-0.038883,-0.140206,-0.095304,-0.082604,0.094886
4,http://dbpedia.org/resource/Interim,1,0.258,0.108478,-0.069382,-0.018091,-0.017771,-0.031568,-0.015339,0.018808,...,0.111549,-0.035377,0.009344,0.008678,0.049789,-0.036075,-0.092665,-0.078730,-0.064595,0.077376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,0.290371,-0.188662,-0.211980,-0.026737,-0.544273,0.097096,0.362069,...,0.250088,0.112760,0.498617,0.298946,0.034676,0.174490,-0.286853,-0.143946,-0.344494,0.414988
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,0.240768,-0.149324,-0.046006,-0.041425,-0.068628,-0.028710,0.056661,...,0.237261,-0.152962,-0.109040,0.008971,0.101864,0.000453,-0.109701,-0.122422,-0.016251,0.101002
635,http://dbpedia.org/resource/Refugee,50,0.507,0.349564,-0.177297,0.006670,0.017165,-0.097819,-0.057915,0.113345,...,0.262677,-0.084342,-0.216944,0.047489,0.045265,-0.008531,-0.116339,-0.084640,-0.064133,0.112939
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,-0.084761,-0.116680,0.550538,0.560814,-0.466635,0.195705,0.292799,...,0.402659,0.264922,-0.218491,0.099624,0.019939,0.105649,-0.142350,0.232590,-0.296790,0.215688
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,0.306742,-0.131736,-0.299866,0.116574,-0.588413,0.145954,0.314122,...,0.287308,-0.034254,0.501102,0.331282,-0.170772,0.149103,-0.566943,-0.132967,-0.179744,0.470595
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.127127,0.095752,-0.285491,-0.109764,-0.221885,0.246013,0.509733,...,0.311846,0.252313,-0.120391,0.108980,-0.078103,0.004863,-0.008975,-0.132159,-0.052691,-0.052647
3,http://dbpedia.org/resource/Interim_leader,1,0.389,0.141770,-0.122628,-0.024322,-0.014976,-0.039883,-0.026032,0.039611,...,0.151385,-0.069491,-0.016031,0.015915,0.038179,-0.038883,-0.140206,-0.095304,-0.082604,0.094886
4,http://dbpedia.org/resource/Interim,1,0.258,0.108478,-0.069382,-0.018091,-0.017771,-0.031568,-0.015339,0.018808,...,0.111549,-0.035377,0.009344,0.008678,0.049789,-0.036075,-0.092665,-0.078730,-0.064595,0.077376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,0.290371,-0.188662,-0.211980,-0.026737,-0.544273,0.097096,0.362069,...,0.250088,0.112760,0.498617,0.298946,0.034676,0.174490,-0.286853,-0.143946,-0.344494,0.414988
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,0.240768,-0.149324,-0.046006,-0.041425,-0.068628,-0.028710,0.056661,...,0.237261,-0.152962,-0.109040,0.008971,0.101864,0.000453,-0.109701,-0.122422,-0.016251,0.101002
635,http://dbpedia.org/resource/Refugee,50,0.507,0.349564,-0.177297,0.006670,0.017165,-0.097819,-0.057915,0.113345,...,0.262677,-0.084342,-0.216944,0.047489,0.045265,-0.008531,-0.116339,-0.084640,-0.064133,0.112939
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,name,doc,weight,v0,v1,v2,v3,v4,v5,v6,...,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199
0,http://dbpedia.org/resource/Democratic_Party_(...,1,0.193,-0.084761,-0.116680,0.550538,0.560814,-0.466635,0.195705,0.292799,...,0.402659,0.264922,-0.218491,0.099624,0.019939,0.105649,-0.142350,0.232590,-0.296790,0.215688
1,http://dbpedia.org/resource/Australian_Senate,1,0.469,0.306742,-0.131736,-0.299866,0.116574,-0.588413,0.145954,0.314122,...,0.287308,-0.034254,0.501102,0.331282,-0.170772,0.149103,-0.566943,-0.132967,-0.179744,0.470595
2,http://dbpedia.org/resource/Brian_Greig,1,1.000,-0.127127,0.095752,-0.285491,-0.109764,-0.221885,0.246013,0.509733,...,0.311846,0.252313,-0.120391,0.108980,-0.078103,0.004863,-0.008975,-0.132159,-0.052691,-0.052647
3,http://dbpedia.org/resource/Interim_leader,1,0.389,0.141770,-0.122628,-0.024322,-0.014976,-0.039883,-0.026032,0.039611,...,0.151385,-0.069491,-0.016031,0.015915,0.038179,-0.038883,-0.140206,-0.095304,-0.082604,0.094886
4,http://dbpedia.org/resource/Interim,1,0.258,0.108478,-0.069382,-0.018091,-0.017771,-0.031568,-0.015339,0.018808,...,0.111549,-0.035377,0.009344,0.008678,0.049789,-0.036075,-0.092665,-0.078730,-0.064595,0.077376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,http://dbpedia.org/resource/Kerry_Nettle,50,1.000,0.290371,-0.188662,-0.211980,-0.026737,-0.544273,0.097096,0.362069,...,0.250088,0.112760,0.498617,0.298946,0.034676,0.174490,-0.286853,-0.143946,-0.344494,0.414988
634,http://dbpedia.org/resource/Right_of_asylum,50,0.279,0.240768,-0.149324,-0.046006,-0.041425,-0.068628,-0.028710,0.056661,...,0.237261,-0.152962,-0.109040,0.008971,0.101864,0.000453,-0.109701,-0.122422,-0.016251,0.101002
635,http://dbpedia.org/resource/Refugee,50,0.507,0.349564,-0.177297,0.006670,0.017165,-0.097819,-0.057915,0.113345,...,0.262677,-0.084342,-0.216944,0.047489,0.045265,-0.008531,-0.116339,-0.084640,-0.064133,0.112939
636,http://dbpedia.org/resource/Mandatory_detentio...,50,0.421,0.280483,-0.224110,-0.082744,0.036705,-0.290722,-0.007814,0.164820,...,0.239666,-0.005728,0.070412,0.100304,0.009112,0.048779,-0.168384,-0.126220,-0.170957,0.232462


In [ ]:
print(results_df)

  Embedding Mode  Dimensions          conf  Pearson Score  Spearman Score  \
0           cbow          50  with_weights       0.037924       -0.040421   
1           cbow         100  with_weights       0.034930       -0.043257   
2           cbow         200  with_weights       0.032786       -0.045945   
3             sg          50  with_weights       0.103162       -0.000709   
4             sg         100  with_weights       0.097167       -0.004475   
5             sg         200  with_weights       0.096744       -0.003191   

   Harmonic Mean  
0       1.227850  
1       0.362889  
2       0.228937  
3      -0.001428  
4      -0.009381  
5      -0.006599  


When use distance_metric='euclidean' without weight

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # Add vectors into DataFrame 'data'
        # Convert the values of the dictionary to a list
        default_value = np.full((dim,), np.nan)
        embedding_list = [embedding_dict.get(entity, default_value) for entity in data['name']]

        embedding_df = pd.DataFrame(embedding_list)

        # Concatenate the original 'data' DataFrame with the 'embedding_df' DataFrame
        data_with_embeddings = pd.concat([data, embedding_df], axis=1)

        # Rename the new column
        new_columns = ['v' + str(i) for i in range(embedding_df.shape[1])]
        col_names = list(data.columns) + new_columns
        data_with_embeddings.columns = col_names
        # Fill in missing entity vectors using the average of other entity vectors within the same document
        # Find out which documents have entities with missing vectors
        docs_with_missing = data_with_embeddings[data_with_embeddings.isnull().any(axis=1)]['doc'].unique()

        for doc in docs_with_missing:
            # Get all entity data in the document
            doc_data = data_with_embeddings[data_with_embeddings['doc'] == doc]

            # Calculate the average of non-missing entity vectors in the document
            average_vector = doc_data.dropna().iloc[:, 3:].mean()

            # Find the entity with the missing vector in the document
            missing_rows = (data_with_embeddings['doc'] == doc) & data_with_embeddings.isnull().any(axis=1)

            # Fill missing vectors with average vector
            data_with_embeddings.loc[missing_rows, new_columns] = average_vector.values

        # 3. Call the Document Similarity evaluation method
        result_dict = train(data_with_embeddings, gold_standard_df, with_weights=False, debugging_mode=True, distance_metric='euclidean')

        # 4. Collect and save results
        results.append({
            'Embedding Mode': mode,
            'Dimensions': dim,
            'conf': result_dict['conf'],
            'Pearson Score': result_dict['pearson_score'],
            'Spearman Score': result_dict['spearman_score'],
            'Harmonic Mean': result_dict['harmonic_mean']
})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('document_similarity_scores_eu.csv', index=False)


Streaming output truncated to the last 5000 lines.
Doc 3 - Doc 4 : distance similarity -5.836779895703465
Doc 3 - Doc 5 : distance similarity -5.845025700386116
Doc 3 - Doc 6 : distance similarity -4.573203300190079
Doc 3 - Doc 7 : distance similarity -5.256865005411371
Doc 3 - Doc 8 : distance similarity -5.609641647361082
Doc 3 - Doc 9 : distance similarity -5.5963877837579785
Doc 3 - Doc 10 : distance similarity -4.367861045170649
Doc 3 - Doc 11 : distance similarity -3.5158028192759585
Doc 3 - Doc 12 : distance similarity -2.913250813280628
Doc 3 - Doc 13 : distance similarity -3.7725473413409825
Doc 3 - Doc 14 : distance similarity -6.982825202061693
Doc 3 - Doc 15 : distance similarity -4.016794050269659
Doc 3 - Doc 16 : distance similarity -2.844088387971251
Doc 3 - Doc 17 : distance similarity -4.217604171164398
Doc 3 - Doc 18 : distance similarity -5.372811210636546
Doc 3 - Doc 19 : distance similarity -3.7921813891196616
Doc 3 - Doc 20 : distance similarity -6.497095198606834

In [ ]:
print(results_df)

  Embedding Mode  Dimensions             conf  Pearson Score  Spearman Score  \
0           cbow          50  without_weights       0.017269       -0.062461   
1           cbow         100  without_weights       0.011464       -0.067759   
2           cbow         200  without_weights       0.013971       -0.065103   
3             sg          50  without_weights       0.044788       -0.036810   
4             sg         100  without_weights       0.034723       -0.047764   
5             sg         200  without_weights       0.038709       -0.042107   

   Harmonic Mean  
0       0.047737  
1       0.027597  
2       0.035578  
3      -0.413334  
4       0.254342  
5       0.959251  


When use distance_metric='cosine' without weight

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # Add vectors into DataFrame 'data'
        # Convert the values of the dictionary to a list
        default_value = np.full((dim,), np.nan)
        embedding_list = [embedding_dict.get(entity, default_value) for entity in data['name']]

        embedding_df = pd.DataFrame(embedding_list)

        # Concatenate the original 'data' DataFrame with the 'embedding_df' DataFrame
        data_with_embeddings = pd.concat([data, embedding_df], axis=1)

        # Rename the new column
        new_columns = ['v' + str(i) for i in range(embedding_df.shape[1])]
        col_names = list(data.columns) + new_columns
        data_with_embeddings.columns = col_names
        # Fill in missing entity vectors using the average of other entity vectors within the same document
        # Find out which documents have entities with missing vectors
        docs_with_missing = data_with_embeddings[data_with_embeddings.isnull().any(axis=1)]['doc'].unique()

        for doc in docs_with_missing:
            # Get all entity data in the document
            doc_data = data_with_embeddings[data_with_embeddings['doc'] == doc]

            # Calculate the average of non-missing entity vectors in the document
            average_vector = doc_data.dropna().iloc[:, 3:].mean()

            # Find the entity with the missing vector in the document
            missing_rows = (data_with_embeddings['doc'] == doc) & data_with_embeddings.isnull().any(axis=1)

            # Fill missing vectors with average vector
            data_with_embeddings.loc[missing_rows, new_columns] = average_vector.values

        # 3. Call the Document Similarity evaluation method
        result_dict = train(data_with_embeddings, gold_standard_df, with_weights=False, debugging_mode=False, distance_metric='cosine')

        # 4. Collect and save results
        results.append({
            'Embedding Mode': mode,
            'Dimensions': dim,
            'conf': result_dict['conf'],
            'Pearson Score': result_dict['pearson_score'],
            'Spearman Score': result_dict['spearman_score'],
            'Harmonic Mean': result_dict['harmonic_mean']
})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('document_similarity_scores_co.csv', index=False)


In [ ]:
print(results_df)

  Embedding Mode  Dimensions             conf  Pearson Score  Spearman Score  \
0           cbow          50  without_weights       0.119882        0.051057   
1           cbow         100  without_weights       0.115280        0.043464   
2           cbow         200  without_weights       0.117055        0.043130   
3             sg          50  without_weights       0.057806       -0.024065   
4             sg         100  without_weights       0.043549       -0.033966   
5             sg         200  without_weights       0.038545       -0.036780   

   Harmonic Mean  
0       0.071614  
1       0.063127  
2       0.063035  
3      -0.082457  
4      -0.308717  
5      -1.605981  


When use distance_metric='cosine' with weight

In [ ]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # Add vectors into DataFrame 'data'
        # Convert the values of the dictionary to a list
        default_value = np.full((dim,), np.nan)
        embedding_list = [embedding_dict.get(entity, default_value) for entity in data['name']]

        embedding_df = pd.DataFrame(embedding_list)

        # Concatenate the original 'data' DataFrame with the 'embedding_df' DataFrame
        data_with_embeddings = pd.concat([data, embedding_df], axis=1)

        # Rename the new column
        new_columns = ['v' + str(i) for i in range(embedding_df.shape[1])]
        col_names = list(data.columns) + new_columns
        data_with_embeddings.columns = col_names
        # Fill in missing entity vectors using the average of other entity vectors within the same document
        # Find out which documents have entities with missing vectors
        docs_with_missing = data_with_embeddings[data_with_embeddings.isnull().any(axis=1)]['doc'].unique()

        for doc in docs_with_missing:
            # Get all entity data in the document
            doc_data = data_with_embeddings[data_with_embeddings['doc'] == doc]

            # Calculate the average of non-missing entity vectors in the document
            average_vector = doc_data.dropna().iloc[:, 3:].mean()

            # Find the entity with the missing vector in the document
            missing_rows = (data_with_embeddings['doc'] == doc) & data_with_embeddings.isnull().any(axis=1)

            # Fill missing vectors with average vector
            data_with_embeddings.loc[missing_rows, new_columns] = average_vector.values
            print(data_with_embeddings)

        # 3. Call the Document Similarity evaluation method
        result_dict = train(data_with_embeddings, gold_standard_df, with_weights=True, debugging_mode=False, distance_metric='cosine')

        # 4. Collect and save results
        results.append({
            'Embedding Mode': mode,
            'Dimensions': dim,
            'conf': result_dict['conf'],
            'Pearson Score': result_dict['pearson_score'],
            'Spearman Score': result_dict['spearman_score'],
            'Harmonic Mean': result_dict['harmonic_mean']
})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('document_similarity_scores_co_we.csv', index=False)


                                                  name  doc  weight        v0  \
0    http://dbpedia.org/resource/Democratic_Party_(...    1   0.193 -3.967533   
1        http://dbpedia.org/resource/Australian_Senate    1   0.469 -2.460789   
2              http://dbpedia.org/resource/Brian_Greig    1   1.000 -2.176809   
3           http://dbpedia.org/resource/Interim_leader    1   0.389 -0.457326   
4                  http://dbpedia.org/resource/Interim    1   0.258 -0.286703   
..                                                 ...  ...     ...       ...   
633           http://dbpedia.org/resource/Kerry_Nettle   50   1.000 -0.226417   
634        http://dbpedia.org/resource/Right_of_asylum   50   0.279 -0.472871   
635                http://dbpedia.org/resource/Refugee   50   0.507 -1.014620   
636  http://dbpedia.org/resource/Mandatory_detentio...   50   0.421       NaN   
637  http://dbpedia.org/resource/Immigration_detention   50   0.238 -0.844549   

           v1        v2    

In [ ]:
print(results_df)

  Embedding Mode  Dimensions          conf  Pearson Score  Spearman Score  \
0           cbow          50  with_weights       0.144763        0.124233   
1           cbow         100  with_weights       0.143887        0.118752   
2           cbow         200  with_weights       0.139760        0.114085   
3             sg          50  with_weights       0.108027        0.052787   
4             sg         100  with_weights       0.102609        0.045795   
5             sg         200  with_weights       0.099982        0.039939   

   Harmonic Mean  
0       0.133715  
1       0.130117  
2       0.125624  
3       0.070919  
4       0.063327  
5       0.057078  


Prepare for Knowledge Graph

In [1]:
from rdflib import Graph, URIRef, Literal, Namespace
import networkx as nx

In [2]:
# Load the graph from the input file
graph = Graph()
graph.parse("/content/drive/My Drive/dbpedia_files/cleaned_LP50_filtered_dbpedia.ttl", format="turtle")

<Graph identifier=Na61c23537a0947a6944ce5435c1aa96c (<class 'rdflib.graph.Graph'>)>

In [3]:
G = nx.DiGraph()  # Create a directed graph

for s, p, o in graph:
    G.add_edge(s, o, predicate=p)

In [4]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G.nodes())
num_edges = len(G.edges())
print("The DBpedia knowledge graph as of 2016-10 has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The DBpedia knowledge graph as of 2016-10 has 4730197 nodes and 14968329 edges.


graph embeddings with SA in BFS

In [5]:
from google.colab import files
uploaded = files.upload()

Saving main.py to main.py
Saving sa_helper.py to sa_helper.py


In [6]:
from main import *
from sa_helper import *

In [7]:
# Count the neighbor node number for each nodes in the graph
neighbor_counts = {node: len(list(G.neighbors(node))) for node in G.nodes()}

In [23]:
subgraph_F_F = spreading_activation_BFS(graph, G, filtered_entities, neighbor_counts,
                                    alpha=0.186, beta=0.006, max_hops=3, extraction_threshold=0,
                                    strict=False, fan_out=True, excl=False, pop=False)

In [24]:
G_sub = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub.add_edge(s, o, predicate=p)

In [25]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub.nodes())
num_edges = len(G_sub.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 653818 nodes and 710867 edges.


In [26]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('BFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [27]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("BFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [30]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # Add vectors into DataFrame 'data'
        # Convert the values of the dictionary to a list
        default_value = np.full((dim,), np.nan)
        embedding_list = [embedding_dict.get(entity, default_value) for entity in data['name']]

        embedding_df = pd.DataFrame(embedding_list)

        # Concatenate the original 'data' DataFrame with the 'embedding_df' DataFrame
        data_with_embeddings = pd.concat([data, embedding_df], axis=1)

        # Rename the new column
        new_columns = ['v' + str(i) for i in range(embedding_df.shape[1])]
        col_names = list(data.columns) + new_columns
        data_with_embeddings.columns = col_names
        # Fill in missing entity vectors using the average of other entity vectors within the same document
        # Find out which documents have entities with missing vectors
        docs_with_missing = data_with_embeddings[data_with_embeddings.isnull().any(axis=1)]['doc'].unique()

        for doc in docs_with_missing:
            # Get all entity data in the document
            doc_data = data_with_embeddings[data_with_embeddings['doc'] == doc]

            # Calculate the average of non-missing entity vectors in the document
            average_vector = doc_data.dropna().iloc[:, 3:].mean()

            # Find the entity with the missing vector in the document
            missing_rows = (data_with_embeddings['doc'] == doc) & data_with_embeddings.isnull().any(axis=1)

            # Fill missing vectors with average vector
            data_with_embeddings.loc[missing_rows, new_columns] = average_vector.values
            print(data_with_embeddings)

        # 3. Call the Document Similarity evaluation method
        result_dict = train(data_with_embeddings, gold_standard_df, with_weights=True, debugging_mode=False, distance_metric='cosine')

        # 4. Collect and save results
        results.append({
            'Embedding Mode': mode,
            'Dimensions': dim,
            'conf': result_dict['conf'],
            'Pearson Score': result_dict['pearson_score'],
            'Spearman Score': result_dict['spearman_score'],
            'Harmonic Mean': result_dict['harmonic_mean']
})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('document_similarity_scores_co_we_BFS.csv', index=False)


                                                  name  doc  weight        v0  \
0    http://dbpedia.org/resource/Democratic_Party_(...    1   0.193 -0.461839   
1        http://dbpedia.org/resource/Australian_Senate    1   0.469 -0.222373   
2              http://dbpedia.org/resource/Brian_Greig    1   1.000 -0.513702   
3           http://dbpedia.org/resource/Interim_leader    1   0.389  0.039859   
4                  http://dbpedia.org/resource/Interim    1   0.258  0.034267   
..                                                 ...  ...     ...       ...   
633           http://dbpedia.org/resource/Kerry_Nettle   50   1.000 -0.645259   
634        http://dbpedia.org/resource/Right_of_asylum   50   0.279  0.162647   
635                http://dbpedia.org/resource/Refugee   50   0.507  0.031213   
636  http://dbpedia.org/resource/Mandatory_detentio...   50   0.421       NaN   
637  http://dbpedia.org/resource/Immigration_detention   50   0.238 -0.008252   

           v1        v2    

In [31]:
print(results_df)

  Embedding Mode  Dimensions          conf  Pearson Score  Spearman Score  \
0           cbow          50  with_weights       0.108362        0.050213   
1           cbow         100  with_weights       0.097214        0.037771   
2           cbow         200  with_weights       0.094511        0.033550   
3             sg          50  with_weights       0.028133       -0.043024   
4             sg         100  with_weights       0.022909       -0.050017   
5             sg         200  with_weights       0.016756       -0.049058   

   Harmonic Mean  
0       0.068626  
1       0.054404  
2       0.049520  
3       0.162570  
4       0.084536  
5       0.050895  


with DFS

In [34]:
subgraph_F_F = spreading_activation_DFS(graph, G, filtered_entities, neighbor_counts, alpha=0.1,
                                        beta=0.009, max_depth=2, extraction_threshold=0,
                                        strict=False, fan_out=True, excl=False, pop=False)

In [35]:
G_sub_ff = nx.DiGraph()  # Create a directed graph

for s, p, o in subgraph_F_F:
    G_sub_ff.add_edge(s, o, predicate=p)

In [36]:
#Exhibihate the size of subgraph：The number of nodes and edges in the subgraph.
num_nodes = len(G_sub_ff.nodes())
num_edges = len(G_sub_ff.edges())
print("The subgraph of DBpedia based on SA has " + str(num_nodes) + " nodes and " + str(num_edges) + " edges.")

The subgraph of DBpedia based on SA has 653817 nodes and 710867 edges.


In [37]:
# Serialize the graph to TTL format
nt_data = subgraph_F_F.serialize(format='nt')

# Save to a TTL file
with open('DFS_subgraph_ff.nt', 'w') as output_file:
    output_file.write(nt_data)

In [38]:
from pandas.core.frame import fmt

# Create a knowledge graph
kg = KG("DFS_subgraph_ff.nt", fmt='nt')

kgentities = kg._entities

In [39]:
# Initialize some variables
results = []

# Define the walker with depth 4 and 500 walks per entity
walker = RandomWalker(4, 500)

# Train the embeddings for different configurations
for mode in ['cbow', 'sg']:
    for dim in [50, 100, 200]:
        # 1. Generate embeddings
        sg = 1 if mode == 'sg' else 0
        embedder = Word2Vec(sg=sg, window=5, negative=25, vector_size=dim, sample=0, ns_exponent=0.75, epochs=5)
        transformer = RDF2VecTransformer(embedder=embedder, walkers=[walker])

        embeddings,_ = transformer.fit_transform(kg, filtered_entities)

        embedding_dict = {}
        for entity, embedding in zip(filtered_entities, embeddings):
            embedding_dict[entity] = embedding

        # Add vectors into DataFrame 'data'
        # Convert the values of the dictionary to a list
        default_value = np.full((dim,), np.nan)
        embedding_list = [embedding_dict.get(entity, default_value) for entity in data['name']]

        embedding_df = pd.DataFrame(embedding_list)

        # Concatenate the original 'data' DataFrame with the 'embedding_df' DataFrame
        data_with_embeddings = pd.concat([data, embedding_df], axis=1)

        # Rename the new column
        new_columns = ['v' + str(i) for i in range(embedding_df.shape[1])]
        col_names = list(data.columns) + new_columns
        data_with_embeddings.columns = col_names
        # Fill in missing entity vectors using the average of other entity vectors within the same document
        # Find out which documents have entities with missing vectors
        docs_with_missing = data_with_embeddings[data_with_embeddings.isnull().any(axis=1)]['doc'].unique()

        for doc in docs_with_missing:
            # Get all entity data in the document
            doc_data = data_with_embeddings[data_with_embeddings['doc'] == doc]

            # Calculate the average of non-missing entity vectors in the document
            average_vector = doc_data.dropna().iloc[:, 3:].mean()

            # Find the entity with the missing vector in the document
            missing_rows = (data_with_embeddings['doc'] == doc) & data_with_embeddings.isnull().any(axis=1)

            # Fill missing vectors with average vector
            data_with_embeddings.loc[missing_rows, new_columns] = average_vector.values
            print(data_with_embeddings)

        # 3. Call the Document Similarity evaluation method
        result_dict = train(data_with_embeddings, gold_standard_df, with_weights=True, debugging_mode=False, distance_metric='cosine')

        # 4. Collect and save results
        results.append({
            'Embedding Mode': mode,
            'Dimensions': dim,
            'conf': result_dict['conf'],
            'Pearson Score': result_dict['pearson_score'],
            'Spearman Score': result_dict['spearman_score'],
            'Harmonic Mean': result_dict['harmonic_mean']
})

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

# Save to a CSV file
results_df.to_csv('document_similarity_scores_co_we_DFS.csv', index=False)


                                                  name  doc  weight        v0  \
0    http://dbpedia.org/resource/Democratic_Party_(...    1   0.193 -2.678546   
1        http://dbpedia.org/resource/Australian_Senate    1   0.469 -1.260415   
2              http://dbpedia.org/resource/Brian_Greig    1   1.000 -0.197257   
3           http://dbpedia.org/resource/Interim_leader    1   0.389 -0.084788   
4                  http://dbpedia.org/resource/Interim    1   0.258 -0.046787   
..                                                 ...  ...     ...       ...   
633           http://dbpedia.org/resource/Kerry_Nettle   50   1.000 -1.052338   
634        http://dbpedia.org/resource/Right_of_asylum   50   0.279 -0.329899   
635                http://dbpedia.org/resource/Refugee   50   0.507 -0.649729   
636  http://dbpedia.org/resource/Mandatory_detentio...   50   0.421       NaN   
637  http://dbpedia.org/resource/Immigration_detention   50   0.238 -0.113616   

           v1        v2    

In [40]:
print(results_df)

  Embedding Mode  Dimensions          conf  Pearson Score  Spearman Score  \
0           cbow          50  with_weights       0.111159        0.056158   
1           cbow         100  with_weights       0.103578        0.045038   
2           cbow         200  with_weights       0.103923        0.047942   
3             sg          50  with_weights       0.028673       -0.039203   
4             sg         100  with_weights       0.019601       -0.054962   
5             sg         200  with_weights       0.019092       -0.049435   

   Harmonic Mean  
0       0.074618  
1       0.062778  
2       0.065615  
3       0.213509  
4       0.060932  
5       0.062212  
